In [1]:
import warnings

# warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import copy
import json
import pandas as pd
import os
import sys
import featuretools as ft
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

sys.path.append("../../")
from Trane import trane as trane
from datetime import datetime, timedelta

/home/pido/miniconda3/envs/alfriend/lib/python3.7/site-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/home/pido/miniconda3/envs/alfriend/lib/python3.7/site-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


In [2]:
# case_folder = 'youtube_pts'
# os.makedirs(case_folder, exist_ok=True)
# os.chdir(case_folder)

### Upload of the dataset and metadata

In [3]:
df = pd.read_csv("./youtube/USvideos.csv", sep=",")
df["trending_date"] = df["trending_date"].apply(
    lambda x: datetime.strptime(x, "%y.%d.%m")
)
df = df.sort_values(by=["trending_date"])
df = df.fillna(0)
meta = trane.TableMeta(json.loads(open("./youtube/meta.json").read()))

### Defining entity column, time column and cutoff strategy

In [4]:
entity = "category_id"
time = "trending_date"
cutoff = "4d"
cutoff_base = pd.Timestamp(datetime.strptime("2017-11-14", "%Y-%m-%d"))
cutoff_end = pd.Timestamp(datetime.strptime("2018-06-14", "%Y-%m-%d"))
cutoff_strategy = trane.FixWindowCutoffStrategy(entity, cutoff, cutoff_base, cutoff_end)

### Generating prediction problems

In [5]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity, time_col=time, cutoff_strategy=cutoff_strategy
)

problems = problem_generator.generate(df, generate_thresholds=True, n_problems=100)

Success/Attempt = 101/163


### Labeling the prediction tasks

In [6]:
problem_label_dict = {}
for p in problems:
    try:
        x = p.execute(df, -1)
        problem_label_dict[str(p)] = x
    except:
        pass

Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_i

Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_id: 16/16 
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████| category_i

In [7]:
fw = trane.FeaturetoolsWrapper(df, entity, time, "youtube")
problem_features_dict = {}

for prob in problem_label_dict:
    problem_features_dict[prob] = fw.compute_features(
        df, problem_label_dict[prob], cutoff
    )

Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100

Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████████████████████████████████████████████████████████
Built 35 features
Elapsed: 00:04 | Progress: 100%|████████████████

In [8]:
problem_features_dict

{'For each <category_id> predict the number of records in next 4d days':                         COUNT(youtube)  MAX(youtube.comment_count)  \
 category_id time                                                     
 1           2017-11-14               8                      7915.0   
             2017-11-18              49                     11910.0   
             2017-11-22              85                     45240.0   
             2017-11-26             134                     47119.0   
             2017-11-30             174                     48348.0   
 ...                                ...                         ...   
 43          2018-05-21              41                      2181.0   
             2018-05-25              45                      2181.0   
             2018-05-29              49                      2181.0   
             2018-06-02              53                      2181.0   
             2018-06-06              57                      2181.0   
 
   